In [3]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import laspy as lp
import pdal
import pandas as pd
import open3d as o3d
from dbfread import DBF

import itertools

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors


# basedir = os.path.dirname(os.getcwd())
basedir = os.path.abspath(os.path.join(os.getcwd() ,"../"))
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')

sys.path.insert(1, _py)
import loads
import lia
import ray as rayt
import lad
import figures

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt
# %matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [1]:
name = 'kiwifruit'

In [2]:
# load files
las = loads.loadlaz(name)
traj = loads.loaddbf(name)

NameError: name 'loads' is not defined

In [5]:
# See LAS columns names
point_format = las.point_format
if False:
    list(point_format.dimension_names)

In [6]:
# Define point cloud with RGB colours
points = np.vstack((las.x, las.y, las.z)).transpose()
colors = np.vstack((las.red, las.green, las.blue)).transpose()

In [7]:
# define and plot a sample of the point cloud

PCsample = (points[:,1] > 0.34*points[:,0] - 3.0) & (points[:,1] < 0.34*points[:,0] + 5)
PCsample &= (points[:,0] < -0.35*points[:,1] - 10) & (points[:,0] > -0.35*points[:,1] - 20)

pointslist = [points[PCsample]]
colours = [colors[PCsample]/2**16]
if False:
    loads.showPCDS(pointslist, colours)

In [8]:
# trajectory
traj = pd.DataFrame(iter(traj)) # to pandas
points_t = np.vstack((traj.x, traj.y, traj.z)).transpose()


In [9]:
# plot the two point cloudsb
if False:
    pointslist = [points[::4], points_t]
    colours = [colors[::4]/2**16, [1, 0, 0]]
    loads.showPCDS(pointslist, colours)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [10]:
# interpolate trajectory with gps time

df = loads.coordsDF(las, traj)

In [11]:
# show sample of beams
if False:
    loads.showbeams(df[15000000:15000300])

In [13]:
if False:
    df1 = df[::100]
    keep = PCsample[::100]

    plt.figure(figsize=(16, 8))

    plt.scatter(df1['x'], df1['y'], s=1, c='gray', alpha=0.3)
    plt.scatter(df1['x'][keep], df1['y'][keep], s=0.3)

    plt.grid()

    plt.xlim(-70, 20)
    plt.ylim(-35,15)

    plt.xlabel(r'$x$', size=20)
    plt.ylabel(r'$y$', size=20)

    plt.show()

## Tree and leaves segmentation

### DBSCAN mask

In [14]:

keep = PCsample & (las.z > 2.5) & (las.z < 7)
Ntot = keep.sum()

db = DBSCAN(eps=0.25).fit(points[keep])

labels = {}

for i in set(db.labels_):
    mask = db.labels_ == i
    perc = 100*mask.sum()/Ntot
    if perc > 1:
        # print(i, perc)
        labels[i] = mask

N = len(list(labels.keys()))

# plot the two point clouds
coltmp = plt.cm.jet(np.linspace(0,1,N))[:,0:3]

if False:
    pointslist = [points[keep][val] for val in labels.values()]
    colours = [list(i) for i in coltmp]
    loads.showPCDS(pointslist, colours)

In [15]:
above = np.zeros(len(points), dtype=bool)
keep = PCsample & (las.z > 2.5) & (las.z < 7)

for val in labels.values():

    above[keep] |= val

keep = PCsample & ~above
rej = PCsample & above

if False:
    pointslist = [points[keep], points[rej]]
    colours = [colors[keep]/2**16, [1, 0, 0]]
    loads.showPCDS(pointslist, colours)

## Percentiles masking

In order to get rid of the stems of kiwifruit trees, first, we take a slide parallel to the stems arange. We have a total of two slides that folow equation:

$$y = 0.34x - 0.5 \\
y = 0.34x + 3.3, $$

each one with width of $0.6$.

In [16]:
keep = PCsample & (las.z < 1.3) & (las.z > 0.4)

plt.figure(figsize=(16, 8))

plt.scatter(np.array(las.x)[keep], np.array(las.y)[keep], c='k', s=0.01)

x = np.linspace(-30, -10, 20)

plt.plot(x, 0.34*x - 0.5, c='r', lw=1, ls='--')
plt.plot(x, 0.34*x + 3.3, c='r', lw=1, ls='--')

for i in [0.3, -0.3]:

    plt.plot(x, 0.34*x - 0.5 + i, c='g', lw=1, ls='-')
    plt.plot(x, 0.34*x + 3.3 + i, c='g', lw=1, ls='-')

plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$y$', size=20)

plt.xlim(-20, -10)

(-20.0, -10.0)

In [17]:
slide1 = (np.array(las.y) > 0.34*np.array(las.x) - 0.5 - 0.3) & (np.array(las.y) < 0.34*np.array(las.x) - 0.5 + 0.3)
slide2 = (np.array(las.y) > 0.34*np.array(las.x) + 3.3 - 0.3) & (np.array(las.y) < 0.34*np.array(las.x) + 3.3 + 0.3)

below = np.zeros(len(points), dtype=bool)

for i in [slide1, slide2]:

    keep = PCsample & i & (las.z > 1.5) & (las.z < 2.7) & ~above

    plt.figure(figsize=(16, 8))
    plt.scatter(np.array(las.x)[PCsample & ~above & i][::1], np.array(las.z)[PCsample & ~above & i][::1], s=0.04, c='k')
    plt.axhline(1.5, lw=1, c='k')
    plt.axhline(2.7, lw=1, c='k')

    res, bcmask = loads.remove_outliers(np.array(las.x)[keep], np.array(las.z)[keep], nbins=100, bounds=(2, 98.0))
    below[keep] |= ~bcmask

    plt.ylim(-1,4)

    plt.xlabel(r'$x$', size=20)
    plt.ylabel(r'$z$', size=20)



In [18]:
# sanity check: check that red dots corresponds to the regions we want to mask out
if False:
    keep = PCsample & ~above & ~below
    rej = PCsample & (above | below)
    pointslist = [points[keep], points[rej]]
    colours = [colors[keep]/2**16, [1, 0, 0]]
    loads.showPCDS(pointslist, colours)

In [19]:
# foliage

foliage = ~above & ~below & (las.z > 1.5) & (las.z < 2.7)
keep = PCsample & foliage
# rej = PCsample & ~foliage
if False:
    pointslist = [points[keep]]
    colours = [colors[keep]/2**16]
    loads.showPCDS(pointslist, colours)

In [20]:
N = PCsample.sum()
samp = PCsample & foliage

print('total # points in sample: %i' %(N))
print('total # points within foliage: %i, \t %.2f %%' %(samp.sum(), 100*samp.sum()/N))

total # points in sample: 1158831
total # points within foliage: 1058672, 	 91.36 %


In [21]:
def segtree(df, leaves, show=False):

    trees = {}
    centres = []
    # keepS = PCsample[::100]
    # PCsample = (points[:,1] > 0.34*points[:,0] - 3.0) & (points[:,1] < 0.34*points[:,0] + 5)
    # PCsample &= (points[:,0] < -0.35*points[:,1] - 10) & (points[:,0] > -0.35*points[:,1] - 20)
    # bins = np.arange(10, 20, 1)
    squares = list(itertools.product(np.arange(-20, -10, 1), np.arange(-3, 5, 1)))

    if show:
        plt.figure(figsize=(20, 10))
        

    for num, (i,j) in enumerate(squares):
        # print(i,j)
        keep = np.ones(len(df['x']), dtype=bool)
        keep &= (df['y'] > 0.34*df['x'] + j) & (df['y'] < 0.34*df['x'] + (j+1))
        keep &= (df['x'] > -0.35*df['y'] + i) & (df['x'] < -0.35*df['y'] + (i+1))
        # print(np.sum(keep))

        trees['tree_%s' %(str(num))] = keep
            
        if show:
            plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.5, label=i)
            p = ((i+0.5) - 0.35 * (j+0.5)) / (1 + (0.35 * 0.34))
            ya = 0.34*(p) + (j+0.5)
            centres.append([num, p, ya])
            # plt.scatter(p,ya, s=20, c='k')

            box = dict(facecolor='green', edgecolor='black', boxstyle='round,pad=0.5', alpha=0.8)
            text = 'tree_%s' %(str(num))

            # x = np.linspace(-18, -12, 5)
            # plt.plot(x, 0.34*x + j + 0.5, c='r', lw=1, ls='--')
            # plt.plot(x, (x - (i + 0.5))/(-0.35), c='r', lw=1, ls='--')

            plt.text(p, ya, text, size=10, bbox=box)

    if show:
        # plt.scatter(df['x'][leaves], df['y'][leaves], s=0.1, c='k')
        plt.xlabel(r'$x$', size=20)
        plt.ylabel(r'$y$', size=20)
        # plt.show()
        
    return trees, centres


In [22]:
leaves = PCsample & foliage
trees, centres = segtree(df, leaves, show=True)

In [23]:
# save centre coordinates of patches
resdir = os.path.join(_data, name, 'lia')
if not os.path.exists(resdir):
    os.makedirs(resdir)
    
df_centres = pd.DataFrame(centres, columns=['tree_id', 'x', 'y'])
df_centres.to_csv(os.path.join(resdir, 'centres.csv'), index=False)

In [24]:
# show the point cloud from leaves of firs tree only
keep = (trees['tree_0']) & (leaves)
# loads.showPCfromDF(df[keep])

if False:
    pointslist = [points[keep]]
    colours = [colors[keep]/2**16]
    loads.showPCDS(pointslist, colours)

## Intensity vs MLS beam trajectory distance

In [25]:
sensors = np.vstack((df['xs'].values, df['ys'].values, df['zs'].values)).transpose()

# get the distances between the PC and the sensor
ab = sensors - points
dist = np.sqrt(np.sum((ab) ** 2, axis=1))
dist

array([2.1927497 , 1.79635658, 2.09363725, ..., 2.5842023 , 2.13051761,
       1.86470625])

In [26]:
keep = PCsample

f = plt.figure(figsize=(10,6))
plt.hist(dist[keep], 50, density=True)
plt.title('Distance between the PC and sensor', size=15)

distbins = {}

for i in [10, 20, 30, 40, 50]:

    distbins['%s-%s.1' %(str(i), str(i))] = np.logical_and(dist[keep] > i, dist[keep] < i+0.1)
    plt.axvline(i, ls='--', color='k')


intensity = np.array(las.intensity[keep])

f = plt.figure(figsize=(10,6))

for key, val in distbins.items():

    print(key, val.sum())

    bins = np.linspace(intensity.min(), intensity.max(), 50)
    plt.hist(intensity[val], bins=bins, histtype='step', lw=2, density=True, label=key)

plt.legend()
plt.title('Intensity', size=15)
    

10-10.1 11718
20-20.1 1829
30-30.1 850
40-40.1 1452
50-50.1 358


Text(0.5, 1.0, 'Intensity')

# `Leaf Inclination Angle` (LIA) estimation

On the contrary to the mock example, here we can not use function `lia.bestfit_pars_la` to get the best-fit parameters...

In [68]:

def run_lia(trees, voxel_size_w, kd3_sr, max_nn, voxel_size_h=0.1, savefig=True, savefig_extra=False):

    # load bestfit results      
    for num, (key, val) in enumerate(trees.items()):

        # if num == 0: # for debug
        if True:

            print('********* %s *********' %(key))

            keep = (val) & (leaves)
            df_ = df[['x', 'y', 'z']][keep]
            points = loads.DF2array(df_)
            # print(points[:,2].min())

            # voxel_size_w = 0.01
            # kd3_sr = 0.15
            # max_nn = 40

            text = '%s=%.4f \n %s=%.4f \n %s=%.4f ' %(
                                    'voxel_size_w', voxel_size_w,
                                    'kd3_sr', kd3_sr,
                                    'max_nn', max_nn)
            # print(text)

            lia.leaf_angle(points, name, key, voxel_size_w, kd3_sr, max_nn, save=True,
                                        savefig=savefig, text=text, voxel_size_h=voxel_size_h, ismock=False,
                                        ylim=0.03, ylimh=0.45, savefig_extra=savefig_extra)

In [69]:
run_lia(trees, voxel_size_w=0.01, kd3_sr=0.15, max_nn=300, 
voxel_size_h=0.05, savefig=False, savefig_extra=True)

********* tree_0 *********
********* tree_1 *********
********* tree_2 *********
********* tree_3 *********
********* tree_4 *********
********* tree_5 *********
********* tree_6 *********
********* tree_7 *********
********* tree_8 *********
********* tree_9 *********
********* tree_10 *********
********* tree_11 *********
********* tree_12 *********
********* tree_13 *********
********* tree_14 *********
********* tree_15 *********
********* tree_16 *********
********* tree_17 *********
********* tree_18 *********
********* tree_19 *********
********* tree_20 *********
********* tree_21 *********
********* tree_22 *********
********* tree_23 *********
********* tree_24 *********
********* tree_25 *********
********* tree_26 *********
********* tree_27 *********
********* tree_28 *********
********* tree_29 *********
********* tree_30 *********
********* tree_31 *********
********* tree_32 *********
********* tree_33 *********
********* tree_34 *********
********* tree_35 *********
**

## Files for Junqi

Following code takes `results_per_height_tree_<ID>.csv` file adds the `LAD` column and break the `values` array to asign a value to a column. Result is stored in Junqi directory with same name.

In [71]:
import glob

for file in glob.glob(os.path.join(_data, name, 'lia', 'results_per_*.csv')):
    # print(file)
    df_ = pd.read_csv(file)
    outfile = os.path.join(_data, name, 'lia', 'junqi', file.split('/')[-1])
    a = df_['values'].str[1:-1]
    keep = [len(i) > 0 for i in a]
    a = a.str.split(expand=True)
    df_['lad'] = np.arange(0, len(a))
    pd.concat([df_['zmin'][keep], df_['lad'][keep], a[keep]], axis=1).to_csv(outfile, index=False) 


Below code is deprecated now...

In [27]:
if False:
    
    import glob

    files = glob.glob(os.path.join(_data, name, 'lia', 'angles*.npy'))
    for file in files:
        outfile = os.path.join(_data, name, 'lia', 'junqi', file.split('/')[-1].split('.')[0]+'.csv')
        pd.DataFrame(np.load(file)).to_csv(outfile, index=False)
        
    for num, (key, val) in enumerate(trees.items()):
        if True:
            keep = (val) & (leaves)
            outfile = os.path.join(_data, name, 'lia', 'junqi', 'heighta_%s.csv' %(key))
            df[['z']][keep].to_csv(outfile, index=False)

# `Leaf Area Density` (LAD) estimation

In [22]:
for key, val in trees.items():

    keep = (val)
    print(key, np.sum(keep))

tree_0 1158831


In [ ]:
downsample = 0.005
voxel_size = 0.15
# to check everything looks fine
show = False
sample = None

In [23]:

POINTS = loads.DF2array(df[['x', 'y', 'z']])
SENSORS = loads.DF2array(df[['xs', 'ys', 'zs']])

def get_rays(downsample, voxel_size, sample=None, show=False):

    if downsample is not None:

        resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
        if not os.path.exists(resdir):
            os.makedirs(resdir)

        outdir = os.path.join(resdir, 'inds.npy')
        if os.path.exists(outdir):
            print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

            inds = np.load(outdir)

            points = POINTS[inds]
            sensors = SENSORS[inds]

        else:

            print('inds not been created yet for donwnsample of %.3f' %(downsample))
            idx = np.random.randint(0, len(df), int(len(df) * downsample))
            inds = np.zeros(len(df), dtype=bool)
            inds[idx] = True

            points = POINTS[inds]
            sensors = SENSORS[inds]

            np.save(outdir, inds)

    else:

        resdir = os.path.join(_data, name, 'lad')
        if not os.path.exists(resdir):
            os.makedirs(resdir)

    if sample is not None:

        idx = np.random.randint(0, len(df), int(sample))
        points = POINTS[idx]
        sensors = SENSORS[idx]

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        m3s = rayt.main(points, sensors, pointsPR, voxel_size, resdir, key, show=show)

In [24]:
def runall(downsample, voxel_size, kbins=None):
    
    if downsample is not None:
        resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
        inds_file = os.path.join(resdir, 'inds.npy')
        inds = np.load(inds_file)
        print('downsample:', downsample)
    else:
        inds = np.ones(len(df), dtype=bool)
        resdir = os.path.join(_data, name, 'lad')

    isfigures = os.path.join(resdir, 'figures')
    if not os.path.exists(isfigures):
        os.makedirs(isfigures)

    print('voxel_size:', voxel_size)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        sensorsPR = SENSORS[inPR]

        m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)

        # _,_,_, m3scount = lad.density_counts(pointsPR, voxel_size)

        # get in down sample boolean array for LPC size
        inds_ = inds[(val) & (leaves)]
        lias, ws = lad.downsample_lia(name, key, inds_)
        voxk = lad.get_voxk(pointsPR, voxel_size)
        bia = lad.get_bia(pointsPR, sensorsPR)
        # meshfile = lad.get_meshfile(name)

        figext = '%s_%s' %(key, str(voxel_size))
        # figext = None
        alphas_k = lad.alpha_k(bia, voxk, lias, ws, resdir, meshfile=None, figext=figext, 
                                klia=False, use_true_lia=False)

        kmax = m3att.shape[2]
        if kbins is None:
            kbins = int(kmax/15)
        print(kbins)

        # Attribute 2 counts per voxel
        # outdir_count = os.path.join(resdir, 'm3count_%s_%s.npy' %(key, str(voxel_size)))
        # m3pcount = np.load(outdir_count)
        # print('******* outdir_count', outdir_count)

        
        # lads_min = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,2], 1)
        # lads_max = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,4], 1)
        lads_mid = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], alpha2=1)
        # lads_mid_w = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1, m3scount=m3scount, m3pcount=m3pcount)
        # lads_mid_counts = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1, m3scount=m3scount, usecounts=True, m3pcount=m3pcount)

        lads_0 = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, alpha2=1)
        # lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax)

        lads = {'Correction Mean':lads_mid, 'No Correction':lads_0} #, 'Correction counts':lads_mid_counts}
        # lads = {'Truth':lads_mesh, 'Correction Mean':lads_mid, 'No Correction':lads_0}
        clai = lad.get_clai(m3att, alphas_k)
        attributes_file = os.path.join(resdir, 'm3s_%s_%s.npy' %(key, str(voxel_size)))
        if os.path.isfile(attributes_file):
            RT = 'Y'
        else:
            RT = 'N'
            
        text = {'tree':key, 'VS':voxel_size, 'DS':downsample, 'RT':RT, 'CLAI':np.round(clai, 3)}
        txt = []
        for key, val in text.items():
            txt.append('%s=%s \n' %(key, str(val)))
        text = (' ').join(txt)

        savefig = os.path.join(resdir, 'figures','LAD_%s.png' %(figext))
        figures.plot_lads(lads, text, savefig=savefig)

    # return m3pcounts, m3scount



In [47]:
downsample = 0.01
voxel_size = 0.1
# to check everything looks fine
show = False
sample = None

get_rays(downsample, voxel_size, sample, show)

inds file already exists for donwnsample of 0.010 at /Users/omar/projects/planttech/data/kiwifruit/lad_0.01/inds.npy
max --> [113, 100, 11]
min --> [0, 0, 0]


50it [00:00, 247.42it/s]

tot vox: 	 138168
voxels hitted: 	 85
Percentage of voxels hitted by beam: 0.00
voxels hitted (OLD): 	 114
Percentage of voxels hitted by beam (OLD): 0.00


## Attribute variation

In [50]:
for DS in [0.005, 0.01, 0.05, 0.1, 0.2]:

    get_rays(downsample=DS, voxel_size=0.1)

inds not been created yet for donwnsample of 0.005
max --> [114, 100, 10]
min --> [0, 0, 0]


196205it [07:08, 457.63it/s] 


tot vox: 	 127765
voxels hitted: 	 79405
Percentage of voxels hitted by beam: 0.62
voxels hitted (OLD): 	 97851
Percentage of voxels hitted by beam (OLD): 0.77
inds not been created yet for donwnsample of 0.010
max --> [114, 101, 11]
min --> [0, 0, 0]


391420it [16:33, 394.03it/s] 


tot vox: 	 140760
voxels hitted: 	 96577
Percentage of voxels hitted by beam: 0.69
voxels hitted (OLD): 	 121262
Percentage of voxels hitted by beam (OLD): 0.86
inds not been created yet for donwnsample of 0.050
max --> [114, 101, 12]
min --> [0, 0, 0]


...

tot vox: 	 153985
voxels hitted: 	 112469
Percentage of voxels hitted by beam: 0.73
voxels hitted (OLD): 	 152480
Percentage of voxels hitted by beam (OLD): 0.99
inds not been created yet for donwnsample of 0.200
max --> [114, 101, 12]
min --> [0, 0, 0]


7131248it [6:38:20, 298.38it/s]  


tot vox: 	 152490
voxels hitted: 	 111331
Percentage of voxels hitted by beam: 0.73
voxels hitted (OLD): 	 152246
Percentage of voxels hitted by beam (OLD): 1.00
inds not been created yet for donwnsample of 0.400
max --> [114, 102, 12]
min --> [0, 0, 0]


354092it [10:39, 554.04it/s]  


KeyboardInterrupt: 

In [59]:
results = []

for downsample in [0.005, 0.01, 0.05, 0.1, 0.2]:
    for voxel_size in [0.1]:

        if downsample is not None:
            resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
            inds_file = os.path.join(resdir, 'inds.npy')
            inds = np.load(inds_file)
            print('downsample:', downsample)
        else:
            inds = np.ones(len(df), dtype=bool)
            resdir = os.path.join(_data, name, 'lad')

        isfigures = os.path.join(resdir, 'figures')
        if not os.path.exists(isfigures):
            os.makedirs(isfigures)

        print('voxel_size:', voxel_size)

        for key, val in trees.items():

            inPR = (val) & (leaves) & (inds)
            pointsPR = POINTS[inPR]
            sensorsPR = SENSORS[inPR]

            m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)

            results.append([downsample, voxel_size, (m3att == 1).sum(), (m3att == 2).sum(), (m3att == 3).sum()])

results = np.array(results)

total = np.array(np.sum(results[:,2:5], axis=1))
nI0 = results[:,2] / total
nP0 = results[:,3] / total
n00 = results[:,4] / total

nI = results[:,2]
nP = results[:,3]
n0 = results[:,4]


fig, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2, 1]}, figsize=(10,22))
colors = plt.cm.jet(np.linspace(0,1,5))

for num, voxel_size in enumerate([0.1]):

    keep = (results[:,1] == voxel_size)
    downsample = results[:,0][keep]
    a0.plot(downsample*100, nP0[keep], marker='*', color=colors[num], label='%s' %(voxel_size))
    a0.plot(downsample*100, nI0[keep], marker='*', ls='--', color=colors[num])
    a0.plot(downsample*100, n00[keep], marker='*', ls=':', color=colors[num])
    
    a1.plot(downsample*100, nI[keep]/(nI[keep]+nP[keep]), marker='*', ls='-', color=colors[num])

text = '$n_{P}(k)$: Solid \n $n_{I}(k)$: Dashed \n $n_{0}(k)$: Dotted'
props = dict(boxstyle='round', facecolor='green', alpha=0.3)
a0.text(20, 0.5, text, fontsize=14, bbox=props)
a0.legend(title='Voxel Size')
a0.set_ylabel(r'$N$', size=20)

# a1.axhline(0.035, ls='--', c='k', lw=2)
a1.set_xlabel(r'Downsample (%)', size=20)
a1.set_ylabel(r'$n_{I}/(n_{I}+n_{P})$', size=20)
    


downsample: 0.005
voxel_size: 0.1
max --> [114, 100, 10]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (115, 101, 11)
ray tracker voxel dimensions: 	 (115, 101, 11)
Number of voxels ocupied by points cloud: 	 4461
Number of voxels ocupied by beam points cloud: 	 79405
Total number of voxels in plant regions: 	 127765
Number of voxels with attribute 1: 	 4461
Number of voxels with attribute 2: 	 79405
Number of voxels with attribute 3: 	 43899
downsample: 0.01
voxel_size: 0.1
max --> [114, 101, 11]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (115, 102, 12)
ray tracker voxel dimensions: 	 (115, 102, 12)
Number of voxels ocupied by points cloud: 	 7700
Number of voxels ocupied by beam points cloud: 	 96577
Total number of voxels in plant regions: 	 140760
Number of voxels with attribute 1: 	 7700
Number of voxels with attribute 2: 	 96577
Number of voxels with attribute 3: 	 36483
downsample: 0.05
voxel_size: 0.1
max --> [114, 101, 12]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (115,

Text(0, 0.5, '$n_{I}/(n_{I}+n_{P})$')

# Point Cloud Resolution (PCR)

In [25]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(POINTS[PCsample])
distances, indices = nbrs.kneighbors(POINTS[PCsample])

In [26]:
dist = distances[:, 1]
dmin, dmax = np.percentile(dist, (0, 99.7))
keep = (dist >= dmin) & (dist <= dmax)
dist = dist[keep]
mean = np.round(np.mean(dist), 3)
median = np.round(np.median(dist), 3)
print(dmin, dmax)

plt.figure(figsize=(10, 6))

bins = np.linspace(dmin, dmax, 80)
plt.hist(dist, bins, density=True)
plt.axvline(mean, ls='--', c='k', label='Mean = %.3f' %(mean))
plt.axvline(median, ls='--', c='g', label='Median = %.3f' %(median))
plt.xlabel('Distance (m)', size=15)

plt.legend()
plt.show()

0.0 0.07123559494865138


# Finding the correct voxel size

In [27]:
def isinds(resdir, downsample):

    outdir = os.path.join(resdir, 'inds.npy')
    if os.path.exists(outdir):
        print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

        inds = np.load(outdir)

        points = POINTS[inds]
        sensors = SENSORS[inds]

    else:

        print('inds not been created yet for donwnsample of %.3f' %(downsample))
        idx = np.random.randint(0, len(df), int(len(df) * downsample))
        inds = np.zeros(len(df), dtype=bool)
        inds[idx] = True

        points = POINTS[inds]
        sensors = SENSORS[inds]

        np.save(outdir, inds)

    return inds

def restmp(downsample, voxel_size, show=True):

    resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)
    inds = isinds(resdir, downsample)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        sensorsPR = SENSORS[inPR]

        density_overall, density, counts, m3scount = lad.density_counts(pointsPR, voxel_size)
        
        # dens_ratio = density / density_overall
        # print('Mean density ration: %.2f' %(np.mean(dens_ratio)))
        # print('Median density ration: %.2f' %(np.median(dens_ratio)))

        if show:

            fig = plt.figure(figsize=(10,6))
            plt.hist(counts, 25, align='left')
            plt.axvline(np.mean(counts), color='k', label='Mean counts = %.2f' %(np.mean(counts)))
            plt.xlabel(r'$n_{i}$', size=20)
            plt.legend()
            plt.show()

    return np.mean(counts)


def ds2vs_first(DS, PCR):

    N = np.cbrt(1/DS)
    VS = N * PCR
    
    return VS

def ds2vs(downsample, points, mean_counts=0, step=0.01, bounds=(1.25, 1.3)):

    # nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(points)
    # distances, indices = nbrs.kneighbors(points)

    minb, maxb = bounds

    # dist = distances[:, 1]
    # dmin, dmax = np.percentile(dist, (0, 99.7))
    # keep = (dist >= dmin) & (dist <= dmax)
    # dist = dist[keep]
    # PCR = np.round(np.mean(dist), 4)
    # print('PCR first guess: %.3f' %(PCR))
    PCR = 0.01

    # first guess for voxel size
    voxel_size = ds2vs_first(downsample, PCR)

    resdir = os.path.join(_data, name, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)
    inds = isinds(resdir, downsample)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]

        j = 0

        while (mean_counts < minb) or (mean_counts > maxb):

            # mean_counts = restmp(downsample, voxel_size, show=False)
            density_overall, density, counts, m3scount = lad.density_counts(pointsPR, voxel_size)
            mean_counts = np.mean(counts)
            
            print(mean_counts, voxel_size, step)

            if mean_counts < minb:
                voxel_size += step

            elif mean_counts > maxb:
                voxel_size -= step

            if np.abs(mean_counts - (maxb + minb)/2) < 0.1:
                step /= 2

    return voxel_size


In [28]:
inPR = (trees['tree_0']) & (leaves)
voxels = {}

for DS in [0.01, 0.1]:

    VS = ds2vs(DS, POINTS[inPR], mean_counts=0, step=0.02, bounds=(1.4, 1.45))
    VS = np.round(VS, 3)
    voxels[DS] = VS

inds file already exists for donwnsample of 0.010 at /Users/omar/projects/planttech/data/kiwifruit/lad_0.01/inds.npy
max --> [245, 218, 24]
min --> [0, 0, 0]
1.0465328467153285 0.046415888336127795 0.02
max --> [171, 152, 17]
min --> [0, 0, 0]
1.1201171875 0.0664158883361278 0.02
max --> [131, 117, 13]
min --> [0, 0, 0]
1.2318615751789976 0.0864158883361278 0.02
max --> [107, 95, 10]
min --> [0, 0, 0]
1.401248812270938 0.1064158883361278 0.02
inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/kiwifruit/lad_0.1/inds.npy
max --> [529, 471, 56]
min --> [0, 0, 0]
1.0513469686666317 0.02154434690031884 0.02
max --> [274, 244, 29]
min --> [0, 0, 0]
1.3282885693137358 0.04154434690031884 0.02
max --> [185, 165, 19]
min --> [0, 0, 0]
1.9569823372475705 0.06154434690031885 0.01
max --> [221, 197, 23]
min --> [0, 0, 0]
1.5931292506222356 0.051544346900318845 0.01
max --> [274, 244, 29]
min --> [0, 0, 0]
1.3282885693137358 0.04154434690031884 0.01
max --> [22

In [29]:

plt.figure(figsize=(8,5))
plt.plot(100*np.array(list(voxels.keys())), np.array(list(voxels.values())), marker='o')
plt.xlabel('Downsampling (%)', size=18)
plt.ylabel('Voxel Size', size=18)

Text(0, 0.5, 'Voxel Size')

In [30]:
for DS in [0.01, 0.1]:

    VS = voxels[DS]
    print(DS, VS)

    runall(downsample=DS, voxel_size=VS, kbins=1)

0.01 0.106
downsample: 0.01
voxel_size: 0.106
max --> [107, 95, 10]
min --> [0, 0, 0]
No ray tracing for tree tree_0 and voxel size 0.106
foliage voxel dimensions: 	 (108, 96, 11)
ray tracker voxel dimensions: 	 (108, 96, 11)
Number of voxels ocupied by points cloud: 	 7362
Number of voxels ocupied by beam points cloud: 	 114048
Total number of voxels in plant regions: 	 114048
Number of voxels with attribute 1: 	 7362
Number of voxels with attribute 2: 	 106686
Number of voxels with attribute 3: 	 0
1
0.1 0.045
downsample: 0.1
voxel_size: 0.045
max --> [253, 226, 27]
min --> [0, 0, 0]
No ray tracing for tree tree_0 and voxel size 0.045
foliage voxel dimensions: 	 (254, 227, 28)
ray tracker voxel dimensions: 	 (254, 227, 28)
Number of voxels ocupied by points cloud: 	 71162
Number of voxels ocupied by beam points cloud: 	 1614424
Total number of voxels in plant regions: 	 1614424
Number of voxels with attribute 1: 	 71162
Number of voxels with attribute 2: 	 1543262
Number of voxels wi

# Compute LAD

In [104]:
restmp(downsample=0.05, voxel_size=0.051, show=True)

inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/kiwifruit/lad_0.05/inds.npy
max --> [223, 199, 23]
min --> [0, 0, 0]


1.297504604748568

In [94]:
for DS in [0.03, 0.05, 0.1]:

    VS = voxels[DS]
    print(DS, VS)
    get_rays(DS, VS)

0.03 0.058
inds file already exists for donwnsample of 0.030 at /Users/omar/projects/planttech/data/kiwifruit/lad_0.03/inds.npy
max --> [196, 175, 20]
min --> [0, 0, 0]


1162636it [1:39:48, 194.14it/s] 


tot vox: 	 728112
voxels hitted: 	 498014
Percentage of voxels hitted by beam: 0.68
voxels hitted (OLD): 	 633414
Percentage of voxels hitted by beam (OLD): 0.87
0.05 0.051
inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/kiwifruit/lad_0.05/inds.npy
max --> [223, 199, 23]
min --> [0, 0, 0]


1918734it [3:10:35, 167.79it/s] 


tot vox: 	 1075200
voxels hitted: 	 747555
Percentage of voxels hitted by beam: 0.70
voxels hitted (OLD): 	 956876
Percentage of voxels hitted by beam (OLD): 0.89
0.1 0.039
inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/kiwifruit/lad_0.1/inds.npy
max --> [292, 260, 31]
min --> [0, 0, 0]


1873458it [3:59:46, 130.23it/s] 


KeyboardInterrupt: 

In [27]:
for DS in [0.01, 0.05]:

    VS = voxels[DS]
    print(DS, VS)

    runall(downsample=DS, voxel_size=VS, kbins=1)

0.01 0.062
downsample: 0.01
voxel_size: 0.062
max --> [183, 163, 18]
min --> [0, 0, 0]
No ray tracing for tree tree_0 and voxel size 0.062
foliage voxel dimensions: 	 (184, 164, 19)
ray tracker voxel dimensions: 	 (184, 164, 19)
Number of voxels ocupied by points cloud: 	 9369
Number of voxels ocupied by beam points cloud: 	 573344
Total number of voxels in plant regions: 	 573344
Number of voxels with attribute 1: 	 9369
Number of voxels with attribute 2: 	 563975
Number of voxels with attribute 3: 	 0
1
0.05 0.031
downsample: 0.05
voxel_size: 0.031
max --> [367, 327, 37]
min --> [0, 0, 0]
No ray tracing for tree tree_0 and voxel size 0.031
foliage voxel dimensions: 	 (368, 328, 38)
ray tracker voxel dimensions: 	 (368, 328, 38)
Number of voxels ocupied by points cloud: 	 47815
Number of voxels ocupied by beam points cloud: 	 4586752
Total number of voxels in plant regions: 	 4586752
Number of voxels with attribute 1: 	 47815
Number of voxels with attribute 2: 	 4538937
Number of voxe

# DTM

In [9]:
POINTS = loads.DF2array(df[['x', 'y', 'z']])
# SENSORS = loads.DF2array(df[['xs', 'ys', 'zs']])

In [10]:
keep = PCsample & (las.z < 0.8)
plt.figure(figsize=(16, 8))
plt.scatter(np.array(las.x)[keep][::1], np.array(las.z)[keep][::1], s=0.04, c='k')
plt.axhline(0.2, lw=1, c='k')

# res, bcmask = loads.remove_outliers(np.array(las.y)[keep], np.array(las.z)[keep], nbins=100, bounds=(0, 90))
# below[keep] |= ~bcmask

# plt.ylim(-1,4)

plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$z$', size=20)

Text(0, 0.5, '$z$')

In [11]:
keep = PCsample & (las.z < 0)
Ntot = keep.sum()

db = DBSCAN(eps=0.3).fit(points[keep])
# db = KMeans(n_clusters = 8, init='k-means++').fit(points[keep])

labels = {}

for i in set(db.labels_):
    mask = db.labels_ == i
    perc = 100*mask.sum()/Ntot
    if perc > 0:
        print(i, perc)
        labels[i] = mask

N = len(list(labels.keys()))

# plot the two point clouds
coltmp = plt.cm.jet(np.linspace(0,1,N))[:,0:3]

pointslist = [points[keep][val] for val in labels.values()]
colours = [list(i) for i in coltmp]
loads.showPCDS(pointslist, colours)

0 100.0
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [12]:
alpha = 0.8
keep = PCsample & (las.z < 0)
pcd = loads.points2pcd(POINTS[keep], colors=[1,0,0])
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(downpcd, alpha)
mesh.compute_vertex_normals()
# pcd = loads.points2pcd(POINTS[keep], colors=[1,0,0])
# downpcd = pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd, mesh.paint_uniform_color([0.5, 0.3, 0.1])], mesh_show_back_face=True)

alpha=0.800
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [15]:
print('filter with average with 1 iteration')
mesh_out = mesh.filter_smooth_simple(number_of_iterations=2)
mesh_out.compute_vertex_normals()
print(mesh)
vertices = np.asarray(mesh_out.vertices)
pcd = loads.points2pcd(vertices, colors=[1,0,0])
mesh_name = os.path.join(_data, name, 'dtm_mesh.obj')
o3d.io.write_triangle_mesh(mesh_name, mesh_out)
o3d.visualization.draw_geometries([pcd, mesh_out], mesh_show_back_face=True)

filter with average with 1 iteration
TriangleMesh with 3802 points and 7602 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [42]:
np.savetxt(os.path.join(_data,'DTM_vertices.csv'), vertices, fmt="%.8f", delimiter=",")

# To-Do

1) Fix algorithm to find `voxel size` from `downsampling`. It currently seems to work for the toy kiwifruit tree it isn't for the real data. Try:
   * Check that the algorithm realy work by computing several `LAD`s for different `downsampling` values using the toy model.
   * Remove outliers of foliage point cloud `FPC` as these might be adding noise to the mean of points per voxel. Try cliping or with a built-in `Open3D` function.
2) Improve `ray tracing` algorithm:
   * find a way to reduce the main sample i.e. `POINTS` and `SENSORS` to keep only the rays that pass trhough the plant region `PR`. It might be not that easy as many other process are involved.
3) Once `LAD` algorithm works. Usea the leaf inclination angles (`LIA`) and the leaf area index (`LAI`) as input parameters to built a realistic model of the kiwifruit trees. One way to do this is using `Blensor` software. Here, we can create a Python script to create the kiwifruit trees with the option of change some of the tree features such as the leaf sizes and leaf angles. Then, in a recursive algorithm, modify these values until we get the same `LIA` and `LAI` distributions from the real LiDAR.

# Dev Zone

In [49]:
tot = len(df)

for DS in [0.005, 0.01, 0.05, 0.1, 0.2, 0.4]:

    idx = np.random.randint(0, len(df), int(len(df) * DS))

    print(DS, len(idx))


0.005 196707
0.01 393414
0.05 1967071
0.1 3934142
0.2 7868284
0.4 15736568
